In [1]:
# Import Dependencies
import numpy as np
import tensorflow as tf
import sys
import tempfile
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio

In [2]:
# TF Ops to Generate MFCC from Audio
with tf.Session() as sess:
    input_ph = tf.placeholder(tf.float32, [512])
    samples = tf.reshape(input_ph, [512, 1])
    spectrogram = contrib_audio.audio_spectrogram(samples, window_size=512, stride=320, magnitude_squared=True)
    mfccs = contrib_audio.mfcc(spectrogram, 16000, dct_coefficient_count=13)

    sess.run([mfccs], feed_dict={input_ph: np.random.random([512])})
    print('session run works')
    
    # Save Audio to MFCC model graph as TFLite Model
    converter = tf.lite.TFLiteConverter(sess.graph_def, input_tensors=[input_ph], output_tensors=[mfccs])
    converter.allow_custom_ops = True
    tflite_model = converter.convert()

session run works


In [3]:
with tempfile.NamedTemporaryFile(delete=False) as fout:
    temp_name = fout.name
    fout.write(tflite_model)
    fout.flush()

In [4]:
# Test Newly created TFLite Model
try:
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=temp_name)
    interpreter.allocate_tensors()
    print('tflite model prepare works')

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    print("Model Input: ", input_details)
    print("Model Output: ", output_details)

    # Test model on random input data.
    for inp in input_details:
        input_data = np.array(np.random.random_sample(inp['shape']), dtype=np.float32)
        interpreter.set_tensor(inp['index'], input_data)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("Inference Output: ", output_data)
finally:
    try:
        os.remove(temp_name)
    except:
        pass

tflite model prepare works
Model Input:  [{'name': 'Placeholder', 'index': 3, 'shape': array([512], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
Model Output:  [{'name': 'Mfcc', 'index': 1, 'shape': array([ 1,  1, 13], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
Inference Output:  [[[20.008898   -4.563293   -1.1163088  -1.0641364  -0.7265215
   -0.61073065 -1.1987216  -0.454337   -0.10931532 -0.08606966
   -0.27914992  0.07555699  0.20666339]]]
